In [ ]:
import os, json
import ssl
import tempfile
import kfp

In [ ]:
URL = os.getenv("DKUBE_URL")  # DKube UI URL
token = os.environ.get("DKUBE_USER_ACCESS_TOKEN")  # DKube OAuth token from UI
ssl.CERT_REQUIRED = ssl.CERT_NONE      # workaround if above URL is not signed by a trusted CA

client = kfp.Client(host=URL + "/pipeline", client_id="1", existing_token=token)

In [ ]:
@kfp.dsl.pipeline(name="test", description="pipeline with 1 echo component")
def test_pipeline():
    kfp.dsl.ContainerOp("echo", image="bash", command=["echo", "hello"])

In [ ]:
# compile
f = tempfile.NamedTemporaryFile(suffix=".zip", delete=False)
kfp.compiler.Compiler().compile(test_pipeline, f.name)

In [ ]:
# upload
pipeline = client.upload_pipeline(f.name, pipeline_name="test1")

In [ ]:
# run
result = client.create_run_from_pipeline_package(f.name, {}, "test run")

In [ ]:
# wait for completion
r=result.wait_for_run_completion(60 * 20)  # wait for completion

In [ ]:
# delete pipeline
client.delete_pipeline(pipeline.id)  # delete the uploaded pipeline